In [33]:
# Imports

import random
import numpy as np
from sklearn.datasets import load_iris
import pandas as pd
import sklearn.model_selection

In [2]:
# Declare constants

NUM_INPUT_NODES = 4
NUM_OUTPUT_NODES = 3
NUM_HIDDEN_NODES = 10
NUM_HIDDEN_LAYERS = 1

In [3]:
class Node:
    """Each node has a weight and bias"""
    def __init__(self, weight, bias):
        self.activation = 0
        self.weight = weight
        self.bias = bias

In [4]:
def relu(x):
    return np.maximum(0, x)

In [5]:
class MLP:
    "Create a multilayer perceptron"
    def __init__(self, n_in, n_out, n_hidden, hidden_layers):
        self.n_hidden_layers = hidden_layers;
        self.nodes = []
        self.create(n_in, n_out, n_hidden, hidden_layers)

    def create(self, n_in, n_out, n_hidden, hidden_layers):
        nodes = []
        input_layer = [Node(0, 0) for _ in range(n_in)]
        nodes.append(input_layer)
        for i in range(hidden_layers):
            nodes.append([Node(random.uniform(0.0, 0.01), random.uniform(0.0, 0.01)) for _ in range(n_hidden)])
        hidden_layer = [Node(random.uniform(0.0, 0.01), random.uniform(0.0, 0.01)) for _ in range(n_out   )]
        nodes.append(hidden_layer)
        self.nodes = nodes

    def forward_pass(self, X_row):
        for i, node in enumerate(self.nodes[0]):
            node.activation = X_row[i]
        for L in range(1, self.n_hidden_layers):
            print("layer: " + L)
            for j in self.nodes[L]:
                j.activation = relu(sum(k.activation * j.weight for k in self.nodes[L - 1]) + j.bias)
        for i, node in enumerate(self.nodes[-1]):
            node.activation = relu(sum(k.activation * node.weight for k in self.nodes[-2]) + node.bias)

In [7]:
# Create MLP

mlp = MLP(NUM_INPUT_NODES, NUM_OUTPUT_NODES, NUM_HIDDEN_NODES, NUM_HIDDEN_LAYERS)

In [20]:
iris = load_iris()

In [21]:
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [22]:
X = iris.data

In [23]:
y = iris.target

In [24]:
X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [25]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [26]:
# FIXME : the data sizes aren't making sense
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.33)

In [27]:
X_train.shape

(100, 4)

In [28]:
X_test.shape

(50, 4)

In [34]:
mlp.forward_pass(X_train[0])

In [35]:
mlp.nodes

[[<__main__.Node at 0x208e74097d0>,

In [36]:
X_train[0]

array([6.7, 3.3, 5.7, 2.5])